In [1]:
from src import create_faiss_vector_store_and_context, medical_kg_prompt, parser


c:\Users\newac\OneDrive\Desktop\Master\final_structure\src\dynamicKG_builder.py:22: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if extractor is "Custom":


In [ ]:
from src.model import get_llamaindex_model, get_llamaindex_model_mini, get_huggingface_embedding_model
from llama_index.core import Settings

llm = get_llamaindex_model_mini()

llm2 = get_llamaindex_model()

embed_model = get_huggingface_embedding_model()
Settings.embed_model = embed_model
Settings.llm = llm2

In [ ]:
nodes = parser.markdownParser(input_dir="../kgdata/")
print(f"Processed {len(nodes)} nodes.")
if nodes:
    print("First node text:", nodes[1].text)

In [26]:
from src import create_faiss_vector_store_and_context
from llama_index.core import  VectorStoreIndex

vector_store, storage_context = create_faiss_vector_store_and_context(nodes)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [31]:
query_engine = vector_index.as_query_engine(similarity_top_k=5, response_mode="tree_summarize")
response = query_engine.query("how do i review an unconcious person?")
print(response)

# If you want to see the source nodes:
for source_node in response.source_nodes:
    print(source_node.node.text)
    print(source_node.node.metadata)
    print(source_node.node)
    print(source_node.score)

    print("--------------------")

To review an unconscious person, follow these steps:

1. Ensure the person is in a safe position. Place them in the recovery position, lying on their side.
2. Check if the person is breathing normally. Look for chest movements and listen for breathing sounds.
3. Continuously monitor the individual while in the recovery position.
4. If breathing is not normal or if they make gurgling sounds or gasp for air, take immediate action.
5. If the person is unconscious but breathing, keep their airway clear. If necessary, tilt their head back gently by placing one hand on the forehead and lifting the chin with the other.
6. Check for normal breathing for no longer than 10 seconds to assess if the chest is rising and falling adequately.
7. If the person is unresponsive and not breathing, prepare to perform emergency procedures as appropriate.
### assessing the level of consciousness
nb! make sure the pat. is breathing
normally. if unresponsive - could it be a
cardiac arrest?
- does the person ha

In [10]:
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor
import nest_asyncio
nest_asyncio.apply()

In [ ]:
#### experiments with dynamic graph extractor
from src.dynamicKG_builder import knowledge_graph_construction
import asyncio

nodes = nodes  # Use a subset for testing, adjust as needed
if not nodes:
    print("No nodes available for testing. Please ensure you have loaded your data correctly.")
# Run the test
kg_index = asyncio.run(knowledge_graph_construction(nodes[:5]))


# if __name__ == "__main__":
#     # Make sure to configure your LLM and embedding model
#     # Settings.llm = OpenAI(model="gpt-3.5-turbo")  # or your preferred LLM
#     # Settings.embed_model = OpenAIEmbedding()      # or your preferred embedding
#     nodes = nodes  # Use a subset for testing, adjust as needed
#     if not nodes:
#         print("No nodes available for testing. Please ensure you have loaded your data correctly.")
#     # Run the test
#     kg_index, retriever = asyncio.run(test_knowledge_graph_construction(nodes[:50]))

#     # Save the graph
#     if kg_index:
#         kg_index.property_graph_store.save_networkx_graph("kg_graph_v1_all.html")
#         print("Knowledge graph saved to 'kg_graph.html'")
#     else:
#         print("Knowledge graph index creation failed.")

#     # Persist the index
#     try:
    
#         kg_index.property_graph_store.persist(persist_path=f"./kg_index_storage_v1/pg_store.json") # SimplePropertyGraphStore persists to a single file
#         print(f"Property graph store persisted to '/pg_store.json'")
#     except Exception as e:
#         print(f"Error persisting property_graph_store: {e}")
    
    

🏥 Testing Knowledge Graph Construction with Medical Emergency Data
📄 5 medical emergency nodes found


Extracting and inferring knowledge graph from text: 100%|██████████| 5/5 [00:11<00:00,  2.38s/it]
Generating embeddings: 100%|██████████| 4/4 [00:02<00:00,  1.77it/s]

✅ Successfully created PropertyGraphIndex!


In [12]:
retriever = kg_index.as_retriever(include_text=True, similarity_top_k=10, response_mode="tree_summarize")
results = retriever.retrieve("help me revive a person using cpr")
results
for i, result in enumerate(results):  # Show top 2
    print(f"  {i}. Score: {result.score:.3f}")
    print(f"     Text: {result.text}...")

  0. Score: 0.779
     Text: Here are some facts extracted from the provided text:

cpr -> REVIVES -> Unconscious adult
Unconscious adult -> NOT_BREATHING_NORMALLY -> cpr
defibrillator -> REQUIRES -> help
defibrillator -> POSITIONED_ON -> person
caller -> MUST_NOT_FETCH -> defibrillator
HLR instructions -> OFFERED_EVEN_IF -> caller believes person is dead
høyspent strøm -> PREVENTS -> touching person
caller -> SHOWS_RESPCT -> opinion on starting HLR
guidance -> OFFERED_TO -> caller
guidance -> CONTINUOUSLY_PROVIDED -> caller
caller -> ASSISTS -> Unconscious adult
children/adolescents -> IN_PRESENCE_OF -> Unconscious adult

## situational guidance & important to ascertain
- help is on the way as i speak to you.
- you must start cpr (reviving the person). i will tell you what to do.
- don’t hang up, put the phone on speaker if you can.
- if there is defibrillator at hand, get someone else to fetch it. check hjertestarterregisteret (the caller must not fetch a defibrillator / aed if alone

In [27]:
retriever = kg_index.as_retriever(include_text=True, similarity_top_k=10, response_mode="tree_summarize")
results = retriever.retrieve("help me revive a person using cpr, i am not trained in cpr")
# results
for i, result in enumerate(results):  # Show top 2
    print(f"  {i}. Score: {result.score:.3f}")
    print(f"     Text: {result.text}...")

  0. Score: 0.762
     Text: Here are some facts extracted from the provided text:

cpr -> REVIVES -> Unconscious adult
Unconscious adult -> NOT_BREATHING_NORMALLY -> cpr
defibrillator -> REQUIRES -> help
defibrillator -> POSITIONED_ON -> person
caller -> MUST_NOT_FETCH -> defibrillator
HLR instructions -> OFFERED_EVEN_IF -> caller believes person is dead
høyspent strøm -> PREVENTS -> touching person
children/adolescents -> IN_PRESENCE_OF -> Unconscious adult
caller -> ASSISTS -> Unconscious adult
guidance -> OFFERED_TO -> caller
guidance -> CONTINUOUSLY_PROVIDED -> caller
caller -> SHOWS_RESPCT -> opinion on starting HLR

## situational guidance & important to ascertain
- help is on the way as i speak to you.
- you must start cpr (reviving the person). i will tell you what to do.
- don’t hang up, put the phone on speaker if you can.
- if there is defibrillator at hand, get someone else to fetch it. check hjertestarterregisteret (the caller must not fetch a defibrillator / aed if alone

In [14]:
### loading the graph from persisted storage
from llama_index.core.indices import PropertyGraphIndex
from llama_index.core.graph_stores import SimplePropertyGraphStore

graph_store = SimplePropertyGraphStore.from_persist_path("./kg_index_storage_v1/pg_store.json")
index = PropertyGraphIndex.from_existing(property_graph_store=graph_store)




In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5, response_mode="tree_summarize")
response = query_engine.query("how do i review an unconcious person? use only the context from the knowledge graph, n o prior knowledge")
print(response)
print("\n\nSource nodes:")
for source_node in response.source_nodes:
    print(source_node.node.text)
    print(source_node.node.metadata)
    print(source_node.node)
    print(source_node.score)

    print("--------------------")

To review an unconscious person, follow these steps:

1. **Assess Responsiveness**: Verify that the person is unconscious and not breathing normally.
2. **Call for Help**: Notify emergency services to ensure professional assistance is on the way.
3. **Position the Person**: Lay them on their back on a firm, flat surface.
4. **Check Airway**: Open the airway by tilting the head back, placing one hand on the forehead and lifting the chin with the other hand.
5. **Check Breathing**: Observe for any signs of normal breathing for no longer than 10 seconds.

If the person is not breathing:

6. **Start CPR**:
   - For adults, begin chest compressions by kneeling beside the chest, placing your hands in the center, and pushing down hard and fast at a rate of 30 compressions followed by two rescue breaths. Repeat this cycle until help arrives or the person shows signs of life.
   - If dealing with a child, after ensuring the airway is clear, give 5 initial breaths through the mouth while ensurin

#### either work with langgraph or start building llamaindex workflows

In [ ]:
# test if we can retrieve the docs here and pass it to langchain

In [29]:
vector_index.vector_store

FaissVectorStore(stores_text=False, is_embedding_query=True)

In [ ]:
# Hybrid extractor using QueryFusionRetriever

from llama_index.core.retrievers import QueryFusionRetriever

# Define individual retrievers
vector_retriever = vector_index.as_retriever(similarity_top_k=4)
kg_retriever = kg_index.as_retriever(similarity_top_k=4)

# Combine retrievers using QueryFusionRetriever
fusion_retriever = QueryFusionRetriever(
    [vector_retriever, kg_retriever],
    similarity_top_k=4,
    mode="reciprocal_rerank",
    use_async=True
)

# Perform retrieval
nodes_with_scores = fusion_retriever.retrieve("how do i review an unconcious person? use only the context from the knowledge graph, n o prior knowledge")
print(f"Retrieved {len(nodes_with_scores)} nodes with scores.")
for node in nodes_with_scores:
    print(node.node.text)
    print(node.node.metadata)
    print(node.score)
    print("--------------------")




Retrieved 4 nodes with scores.
### assessing the level of consciousness
nb! make sure the pat. is breathing
normally. if unresponsive - could it be a
cardiac arrest?
- does the person have speech problems? confused? single words only? incomprehensible babbling?
- can s/he move both arms and legs on demand?
- are the person’s eyes open? does s/he open them when you say something?
- does the person open his/her eyes if shaken gently?
{'file_path': 'c:\\Users\\newac\\OneDrive\\Desktop\\Master\\final_structure\\..\\kgdata\\27.md', 'file_name': '27.md', 'file_size': 14318, 'creation_date': '2025-04-22', 'last_modified_date': '2025-04-24', 'header_path': '/27 Stroke symptoms\r/Related Questions \r/'}
0.04893312516263336
--------------------
### if yes
- place him / her in the recovery position (on his / her side).
- check if the person is breathing normally.
- the person must be monitored continuously when in the recovery position.
- tell me immediately if the person stops breathing, makes g